In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [2]:
from typing import Annotated # annotated는 타입 힌트를 사용할 때 사용하는 함수
from typing_extensions import TypedDict # TypedDict는 딕셔너리 타입을 정의할 때 사용하는 함수

from langgraph.graph.message import add_messages

# 상태 정의
class State(TypedDict):	
    messages: Annotated[list[str], add_messages]

In [3]:
from langgraph.graph import StateGraph

# 그래프 구성
graph_builder = StateGraph(State)

In [4]:
from langchain_core.tools import tool
from datetime import datetime
import pytz
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# 도구 함수 정의
@tool
def get_current_time(timezone: str, location: str) -> str:
    """현재 시각을 반환하는 함수."""
    try:
        tz = pytz.timezone(timezone)
        now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        result = f'{timezone} ({location}) 현재시각 {now}'
        # print(result)
        return result
    except pytz.UnknownTimeZoneError:
        return f"알 수 없는 타임존: {timezone}"
    
@tool
def get_web_search(query: str, search_period: str='m') -> str:
    """
    웹 검색을 수행하는 함수.
    """
    wrapper = DuckDuckGoSearchAPIWrapper(
        # region="kr-kr", 
        time=search_period
    )

    print('\n-------- WEB SEARCH --------')
    print(query)
    print(search_period)

    search = DuckDuckGoSearchResults(
        api_wrapper=wrapper,
        # source="news",
        results_separator=';\n'
    )

    searched = search.invoke(query)
    
    for i, result in enumerate(searched.split(';\n')):
        print(f'{i+1}. {result}')
    
    return searched

# 도구 바인딩
tools = [get_current_time, get_web_search]

In [5]:
tools[0].invoke({"timezone": "Asia/Seoul", "location": "부산"})

'Asia/Seoul (부산) 현재시각 2025-09-21 17:43:56'

In [6]:
query = "2025년 KT 소액결제 사건의 원인은?"

In [7]:
tools[1].invoke({"query": query, "search_period": "m"})


-------- WEB SEARCH --------
2025년 KT 소액결제 사건의 원인은?
m
1. snippet: KT 위즈 구단은 25일 2025 시즌 연봉 재계약 대상자 64명과의 계약을 마무리했다고 공식적으로 발표했다., title: KT 위즈 구단은 25일 2025시즌 연봉 재계약 대상자, link: https://totosite.one/2025/01/the-kt-wiz-organization-officially-announced-today-that-it-has-finalized-contracts-with-64-players-for-the-2025-season/
2. snippet: KT Foiling dévoile sa gamme 2025 avec des shapes totalement repensés et cinq nouvelles constructions hautes performances : Carbon, Pro Carbon ..., title: KT Foiling Boards 2025 – Nouvelle génération de planches, link: https://blog.swelladdiction.com/2025/04/20/planches-foil-kt-foiling-2025/
3. snippet: Home › Concerts › United Kingdom › London › KT Tunstall - Royal Albert Hall - Jun 23, 2025 ... for KT Tunstall in London, Jun 23, 2025, title: KT Tunstall, Royal Albert Hall, Jun 23, 2025 Tickets, London,, link: https://www.jambase.com/show/kt-tunstall-royal-albert-hall-20250623
4. snippet: 2025 , KT Wiz (Unrestricted Free Agent / 자유선발) ... MyKBO Stats v2 —

'snippet: KT 위즈 구단은 25일 2025 시즌 연봉 재계약 대상자 64명과의 계약을 마무리했다고 공식적으로 발표했다., title: KT 위즈 구단은 25일 2025시즌 연봉 재계약 대상자, link: https://totosite.one/2025/01/the-kt-wiz-organization-officially-announced-today-that-it-has-finalized-contracts-with-64-players-for-the-2025-season/;\nsnippet: KT Foiling dévoile sa gamme 2025 avec des shapes totalement repensés et cinq nouvelles constructions hautes performances : Carbon, Pro Carbon ..., title: KT Foiling Boards 2025 – Nouvelle génération de planches, link: https://blog.swelladdiction.com/2025/04/20/planches-foil-kt-foiling-2025/;\nsnippet: Home › Concerts › United Kingdom › London › KT Tunstall - Royal Albert Hall - Jun 23, 2025 ... for KT Tunstall in London, Jun 23, 2025, title: KT Tunstall, Royal Albert Hall, Jun 23, 2025 Tickets, London,, link: https://www.jambase.com/show/kt-tunstall-royal-albert-hall-20250623;\nsnippet: 2025 , KT Wiz (Unrestricted Free Agent / 자유선발) ... MyKBO Stats v2 — Build 585 ( 2025 -09-03) Powered by Elixir and the Phoenix 

In [8]:
model_with_tools = model.bind_tools(tools) # GPT 언어모델에 도구 연결

def generate(state: State):
    return {"messages": model_with_tools.invoke(state["messages"])}

graph_builder.add_node("generate", generate)

In [9]:
# import json
# from langchain_core.messages import ToolMessage

# class BasicToolNode:
#     """
#     도구를 실행하는 노드 클래스입니다. 마지막 AIMessage에서 요청된 도구를 실행합니다.
#     """

#     def __init__(self, tools: list) -> None:
#         self.tools_by_name = {tool.name: tool for tool in tools} # 딕셔너리 컴프리헨션(Dictionary Comprehension)

#     def __call__(self, inputs: dict):
#         # inputs에 messages가 있으면 messages를 가져오고 없으면 빈 리스트를 가져옵니다.
#         if messages := inputs.get("messages", []): # 왈러스(walrus) 연산자 사용
#             message = messages[-1]
#         else:
#             raise ValueError("No message found in input")
#         outputs = []
#         for tool_call in message.tool_calls:
#             tool_result = self.tools_by_name[tool_call["name"]].invoke(
#                 tool_call["args"]
#             )
#             outputs.append(
#                 ToolMessage(
#                     content=json.dumps(tool_result),
#                     name=tool_call["name"],
#                     tool_call_id=tool_call["id"],
#                 )
#             )
#         return {"messages": messages + outputs}

In [10]:
# # * Case1
# tool_node = BasicToolNode(tools=tools)
# graph_builder.add_node("tools", tool_node)

In [11]:
# * Case2
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

In [12]:
from langgraph.graph import START, END

def route_tools(state: State):
    """
    마지막 메시지에 도구 호출이 있는 경우 ToolNode로 라우팅하고,
    그렇지 않은 경우 END로 라우팅하기 위해 conditional_edge에서 사용합니다.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"tool_edge 입력 상태에서 메시지를 찾을 수 없습니다: {state}")
        
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END

In [13]:
graph_builder.add_edge(START, "generate")

graph_builder.add_conditional_edges(
    "generate",
    route_tools,
    {"tools": "tools", END: END},
)
# 도구가 호출될 때마다 다음 단계를 결정하기 위해 챗봇으로 돌아갑니다.
graph_builder.add_edge("tools", "generate")

In [14]:
graph = graph_builder.compile()

In [15]:
graph.get_graph().print_ascii()

        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
         +----------+         
         | generate |         
         +----------+         
          .         *         
        ..           **       
       .               *      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 


In [16]:
from langchain_core.messages import AIMessageChunk, HumanMessage

inputs = [HumanMessage(content="지금 부산 몇시야?")]

gathered = None

for msg, metadata in graph.stream({"messages": inputs}, stream_mode="messages"):
    if isinstance(msg, AIMessageChunk):
        print(msg.content, end='')

        if gathered is None:
            gathered = msg
        else:
            gathered = gathered + msg

지금 부산은 2025년 9월 21일 17시 44분입니다.


In [17]:
gathered

AIMessageChunk(content='지금 부산은 2025년 9월 21일 17시 44분입니다.\n', additional_kwargs={'function_call': {'name': 'get_current_time', 'arguments': '{"location": "\\ubd80\\uc0b0", "timezone": "Asia/Seoul"}'}}, response_metadata={'finish_reason': 'STOPSTOP', 'model_name': 'gemini-2.0-flashgemini-2.0-flash', 'safety_ratings': []}, id='run--2e6044d2-3f77-494e-93a1-495d68a929bc', tool_calls=[{'name': 'get_current_time', 'args': {'location': '부산', 'timezone': 'Asia/Seoul'}, 'id': '05b61c7c-c9bf-4e04-a55d-1c0abb033abd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 152, 'output_tokens': 41, 'total_tokens': 254, 'input_token_details': {'cache_read': 0}}, tool_call_chunks=[{'name': 'get_current_time', 'args': '{"location": "\\ubd80\\uc0b0", "timezone": "Asia/Seoul"}', 'id': '05b61c7c-c9bf-4e04-a55d-1c0abb033abd', 'index': None, 'type': 'tool_call_chunk'}])

In [18]:
from langchain_core.messages import AIMessageChunk, SystemMessage

inputs = [
    SystemMessage(content=f"""
        너는 신문기자이다. 
        최근 {query}에 대해 비판하는 심층 분석 기사를 쓰려고 한다.
    """),
    HumanMessage(content="""
        - 최근 어떤 이슈가 있는지 검색하고, 사람들이 제일 관심있어 할만한 주제를 선정하고, 왜 선정했는지 말해줘. 
        - 그 내용으로 원고를 작성하기 위한 목차를 만들고, 목차 내용을 채우기 위해 추가로 검색할 내용을 리스트로 정리해봐. 
        - 검색할 리스트를 토대로 재검색을 한다. 
        - 목차에 있는 내용을 작성하기 위해 더 검색이 필요한 정보가 있는지 확인하고, 있다면 추가로 검색해라.
        - 검색된 결과에 원하는 정보를 찾지 못했다면 다른 검색어로 재검색해도 좋다. 
        
        더 이상 검색할 내용이 없다면, 조선일보 신문 기사 형식으로 최종 기사를 작성하라.
        제목, 부제, 리드문, 본문 의 구성으로 작성하라. 본문 내용은 심층 분석 기사에 맞게 구체적이고 깊이 있게 작성해야 한다.
    """)
]

gathered = None

for msg, metadata in graph.stream({"messages": inputs}, stream_mode="messages"):
    if isinstance(msg, AIMessageChunk):
        # print(msg.content, end='')

        if gathered is None:
            gathered = msg
        else:
            gathered = gathered + msg


-------- WEB SEARCH --------
2025년 KT 소액결제
3개월
1. snippet: 과학기술정보통신부는 배경훈 장관이 11일 KT 무단 소액결제 사건과 관련해 서울 종로구 KT 광화문 사옥을 방문해 조치 현황을 점검하고 국민 불안 최소화를 당부했다고 밝혔다.사진은 10일 서울 한 KT 대리점 모습. 2025.9.10 cityboy@yna.co.kr., title: KT 무단 소액결제 사태…과기부 장관 직접 점검 나섰다 | 연합뉴스, link: https://www.yna.co.kr/view/AKR20250911080500017?section=society/accident
2. snippet: 범인 체포: 2025 년 9월 17일, 경기남부경찰청은 KT 무단 소액결제 사건의 용의자로 조선족 2명을 긴급 체포했다는 기사가 나왔다., title: KT 소액결제 범인 체포 (feat 애플계정과 롯데카드 추가로 털림) feat., link: https://www.ilbe.com/view/11597596556
3. snippet: 김나인 2025. 9. 9. 14:55.과학기술정보통신부는 KT 가입자 무단 소액결제 사건과 관련, 조사를 위해 정보보호네트워크정책관을 단장으로 하는 민관합동조사단을 구성해 현장조사 등 신속한 원인조사에 착수했다고 9일 밝혔다., title: KT 소액결제 해킹 피해 확산에…과기정통부, 민관합동조사단 가동, link: https://v.daum.net/v/20250909145547820
4. snippet: 1. 2025 년 KT 해킹은 ‘가짜 기지국’이라는 신종 수법으로, 사용자의 주의만으로는 막기 어려워 소액결제 원천 차단이 필수입니다., title: KT 해킹 사태, 내 돈은 안전할까? SKT, LG, KT 소액결제 차단 2분 만에..., link: https://itprism.co.kr/kt-소액결제-차단/


In [19]:
gathered

AIMessageChunk(content="알겠습니다. 2025년 KT 소액결제 사건에 대한 심층 분석 기사를 작성하기 위해 다음과 같은 단계를 따르겠습니다.\n\n**1. 최신 이슈 검색 및 주제 선정**\n\n먼저, 최근 이슈를 검색하여 사람들이 가장 관심을 가질 만한 주제를 선정하고, 그 이유를 설명하겠습니다.\n\n최근 KT 소액결제 사건이 이슈이며, 특히 '가짜 기지국'이라는 신종 수법이 사용되었다는 점, 그리고 정부의 조사단 구성 및 장관의 현장 점검이 있었다는 점이 주목할 만합니다. 사람들은 자신의 돈이 안전한지, 그리고 이러한 해킹을 어떻게 막을 수 있는지에 가장 큰 관심을 가질 것이라고 판단했습니다.\n\n**목차**\n\n1.  **머리말**: KT 소액결제 사건 발생 및 사회적 파장\n2.  **사건 개요**:\n    *   2025년 KT 소액결제 사건의 구체적인 내용 및 피해 규모\n    *   '가짜 기지국' 수법의 원리 및 작동 방식 상세 분석\n3.  **원인 분석**:\n    *   KT의 보안 시스템 취약점 심층 분석\n    *   '가짜 기지국' 공격에 취약한 이유\n    *   과거 유사 사례와 비교 분석\n4.  **대응 현황**:\n    *   정부의 민관합동조사단 조사 내용 및 결과\n    *   KT의 자체적인 대응책 및 보상 방안\n    *   피해자들의 집단 소송 움직임\n5.  **문제점 및 개선 방안**:\n    *   현재 소액결제 시스템의 구조적 문제점\n    *   '가짜 기지국' 공격에 대한 기술적/제도적 방어책\n    *   소비자 보호를 위한 정책 제언\n6.  **결론**: KT 소액결제 사건의 교훈 및 향후 과제\n\n**추가 검색 내용**\n\n1.  가짜 기지국 원리\n2.  KT 보안 시스템 현황\n3.  소액결제 시스템 문제점\n4.  국내외 유사 해킹 사례\n5.  관련 법규 및 제도\n6.  KT의 과거 해킹 사례 및 대응\n7.  소액결제 피해자 구제 방안\n8. 

In [20]:
print(gathered.content)

알겠습니다. 2025년 KT 소액결제 사건에 대한 심층 분석 기사를 작성하기 위해 다음과 같은 단계를 따르겠습니다.

**1. 최신 이슈 검색 및 주제 선정**

먼저, 최근 이슈를 검색하여 사람들이 가장 관심을 가질 만한 주제를 선정하고, 그 이유를 설명하겠습니다.

최근 KT 소액결제 사건이 이슈이며, 특히 '가짜 기지국'이라는 신종 수법이 사용되었다는 점, 그리고 정부의 조사단 구성 및 장관의 현장 점검이 있었다는 점이 주목할 만합니다. 사람들은 자신의 돈이 안전한지, 그리고 이러한 해킹을 어떻게 막을 수 있는지에 가장 큰 관심을 가질 것이라고 판단했습니다.

**목차**

1.  **머리말**: KT 소액결제 사건 발생 및 사회적 파장
2.  **사건 개요**:
    *   2025년 KT 소액결제 사건의 구체적인 내용 및 피해 규모
    *   '가짜 기지국' 수법의 원리 및 작동 방식 상세 분석
3.  **원인 분석**:
    *   KT의 보안 시스템 취약점 심층 분석
    *   '가짜 기지국' 공격에 취약한 이유
    *   과거 유사 사례와 비교 분석
4.  **대응 현황**:
    *   정부의 민관합동조사단 조사 내용 및 결과
    *   KT의 자체적인 대응책 및 보상 방안
    *   피해자들의 집단 소송 움직임
5.  **문제점 및 개선 방안**:
    *   현재 소액결제 시스템의 구조적 문제점
    *   '가짜 기지국' 공격에 대한 기술적/제도적 방어책
    *   소비자 보호를 위한 정책 제언
6.  **결론**: KT 소액결제 사건의 교훈 및 향후 과제

**추가 검색 내용**

1.  가짜 기지국 원리
2.  KT 보안 시스템 현황
3.  소액결제 시스템 문제점
4.  국내외 유사 해킹 사례
5.  관련 법규 및 제도
6.  KT의 과거 해킹 사례 및 대응
7.  소액결제 피해자 구제 방안
8.  가짜 기지국 탐지 기술
9.  통신사 보안 투자 현황
10. 정부의 통신 보안 정책

